In [ ]:
%load_ext autoreload
%autoreload
%load_ext line_profiler

In [ ]:
import os as _os
_os.chdir(_os.environ['PROJECT_ROOT'])
print(_os.path.realpath(_os.path.curdir))

In [ ]:
from lib.saa_prototype import *

In [ ]:
import graph_tool as gt
import numpy as np

In [ ]:
nucleotide_sequences = []
links = []
lengths = []
# with open('data/genbank/ecoli.mg1655.bcalm-k191.gfa') as f:
# with open('data/genbank/ecoli.o121h19.bcalm-k191.gfa') as f:
# with open('data/genbank/bdorei.dsm17855.bcalm-k191.gfa') as f:
# with open('data/both_species.bcalm-k191.gfa') as f:
with open('data/both_strains.bcalm-k191.gfa') as f:
    for line in f:
        if line.startswith('S'):
            _, idx, nucl, *_ = line.split()
            nucleotide_sequences.append(nucl)
            lengths.append(len(nucl))
        elif line.startswith('L'):
            _, left_idx, left_sign, right_idx, right_sign, _ = line.split()
            left_idx = int(left_idx)
            right_idx = int(right_idx)
            left_reverse = left_sign == '-'
            right_reverse = right_sign == '-'
            links.append(((left_idx, left_reverse), (right_idx, right_reverse)))

In [ ]:
num_nucleotide_sequences = len(nucleotide_sequences)

In [ ]:
print(len(nucleotide_sequences))
nucleotide_sequences[:10]

In [ ]:
print(len(links))
links[:10]

In [ ]:
def f_idx(i):
    return i * 2 + 0


def r_idx(i):
    return i * 2 + 1


def idx(i, reverse):
    return i * 2 + reverse

In [ ]:
edges = []
for (left_i, left_reverse), (right_i, right_reverse) in links:
    edges.append((
        idx(left_i, left_reverse),
        idx(right_i, right_reverse)
    ))
    # It's not clear why the reverse edge is already
    # in the GFA file, but it is.
    edges.append((
        idx(right_i, not right_reverse),
        idx(left_i, not left_reverse)
    ))
    
print(len(edges))
edges = list(set(edges))
print(len(edges))
edges[:10]

In [ ]:
g = gt.Graph()
g.add_edge_list(edges)
g.vp['length'] = g.new_vertex_property('int', val=0)
g.vp['sequence'] = g.new_vertex_property('object', val=[])
g.vp['depth'] = g.new_vertex_property('vector<float>', val=[0])
g.gp['nsample'] = g.new_graph_property('int', val=1)
for i, l in enumerate(lengths):
    g.vp.length[f_idx(i)] = l
    g.vp.length[r_idx(i)] = l
    g.vp.depth[f_idx(i)] = np.array([1])
    g.vp.depth[r_idx(i)] = np.array([1])
    g.vp.sequence[f_idx(i)] = [f_idx(i)]
    g.vp.sequence[r_idx(i)] = [r_idx(i)]
    
gg = [g]
gg

In [ ]:
draw_graph(g, output_size=(2000, 2000))

In [ ]:
g.vp.length.a[[65, 670]]

In [ ]:
gg.append(gg[-1].copy())
mutate_compress_all_unitigs(gg[-1])
assert gg[0].num_edges() == gg[-1].num_edges(), "Compressing the raw graph SHOULD be a no-op. "
# FIXME: Apparently this isn't actually true?

assert (depth_matrix(gg[0]) == depth_matrix(gg[-1])).all()

In [ ]:
bins = np.logspace(0, 7, num=51)
plt.hist(gg[-1].vp.length.a, bins=bins)
plt.yscale('log')
plt.xscale('log')
None

In [ ]:
x = (pd.Series(gg[-1].vp.length.a).sort_values(ascending=False).cumsum() / 2)
plt.plot(np.arange(1, len(x) + 1) / 2, x.values)
plt.xscale('log')

In [ ]:
description = describe_nodes(gg[-1])

In [ ]:
description.sort_values('length', ascending=False)